In [1]:
import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device =  -1
print(device)

0


In [2]:
# specify transformer name
# used to load pipeline and tokenizer
transformer_name = "bert-base-cased"

In [3]:
# load pipeline
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model=transformer_name, device=device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
# load tokenizer
from transformers import AutoTokenizer
from datasets import Dataset
tokenizer = AutoTokenizer.from_pretrained(transformer_name)
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True)

In [5]:
# load all data
import pandas as pd
labels = open('data/classes.txt').read().splitlines()
all_df = pd.read_csv("data/belief_benchmark.csv")
all_df

,text,sentiment,label
0,Also use of chemicals and machinery on their p...,"NEGATIVE, NEGATIVE",1
1,Sources believe that twenty-five to 30 percent...,UNDETERMINED,0
2,Although these government-led initiatives are ...,UNDETERMINED,0
3,"In other projects of this kind, it is not alwa...",POSITIVE,2
4,Many African policy makers evidently believe t...,POSITIVE,2
...,...,...,...
358,Similar initiatives have and can be initiated ...,POSITIVE,2
359,Collective sales account for an average of 64 ...,UNDETERMINED,0
360,Individual sales account for slightly lower vo...,"UNDETERMINED, UNDETERMINED, UNDETERMINED",0
361,Modernization of the rice value chain The VC t...,UNDETERMINED,0


In [6]:
# create dataset from all data
from datasets import Dataset

dataset = Dataset.from_pandas(all_df)
#dataset.save_to_disk("data/belief_dataset")
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

In [7]:
# load and create dataset from holdout data
holdout_df = df = pd.read_csv("data/belief_benchmark_holdout.csv")
holdout_ds = Dataset.from_pandas(holdout_df)
holdout_ds = holdout_ds.map(tokenize, batched=True)

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

In [8]:
# run prediction and save predicted label as index into labels array
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

labels = ["neither", "negative", "positive", "both"]
pred_i_list = []
# use KeyDataset(dataset, ...) for results on all data
# use KeyDataset(holdout_ds, ...) for results on holdout
for pred in tqdm(classifier(KeyDataset(dataset, "text"), batch_size=6, candidate_labels=labels)):
    pred_i = labels.index(pred["labels"][0])
    pred_i_list.append(pred_i)

  0%|          | 0/61 [00:00<?, ?it/s]

In [9]:
# print classification report
from sklearn.metrics import classification_report

y_true = dataset["label"]
y_pred = pred_i_list
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

     neither       0.41      0.46      0.43       122
    negative       0.21      0.32      0.25        65
    positive       0.48      0.20      0.28       147
        both       0.05      0.10      0.06        29

    accuracy                           0.30       363
   macro avg       0.29      0.27      0.26       363
weighted avg       0.37      0.30      0.31       363



In [10]:
# print true vs predicted label idxs, save lists as pickle
print(f"y_true: {y_true}")
print(f"y_pred: {y_pred}")
import pickle
pickle.dump(y_pred, open(f"few_shot_results/y_pred_zero_all", "wb"))
pickle.dump(y_true, open(f"few_shot_results/y_true_zero_all", "wb"))


y_true: [1, 0, 0, 2, 2, 3, 1, 0, 3, 2, 2, 2, 1, 2, 0, 1, 1, 2, 0, 3, 3, 0, 2, 2, 1, 2, 2, 2, 1, 2, 3, 0, 2, 0, 2, 2, 2, 3, 1, 0, 2, 2, 0, 0, 1, 2, 2, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 0, 2, 1, 2, 0, 1, 2, 2, 1, 2, 2, 2, 0, 0, 3, 1, 0, 0, 2, 0, 2, 3, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 1, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 2, 1, 0, 2, 0, 0, 2, 1, 2, 0, 2, 2, 2, 0, 0, 2, 3, 0, 2, 0, 0, 0, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 0, 1, 2, 1, 1, 0, 1, 1, 0, 2, 0, 0, 0, 1, 2, 2, 0, 1, 1, 2, 1, 0, 2, 0, 2, 2, 2, 1, 3, 0, 3, 2, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 2, 2, 1, 3, 2, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 3, 3, 0, 3, 0, 0, 0, 0, 2, 3, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 3, 2, 3, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 1, 2, 2, 2, 2, 3, 2, 2, 0, 2, 3, 2, 2, 0, 1, 2, 0, 2, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 3, 0, 2, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 0, 1, 2, 1, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0

In [11]:
import gc
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()

125